# 부산광역시 문화시설 위도와 경도 좌표 추가
    - 도로명 주소를 통해 위도와 경도로 변환하여 csv파일 저장

In [1]:
import pandas as pd 
import requests 
import time
import random

apikey = 'BBCE5248-AEAB-3BE6-B763-84E29128D063' # API 키를 여기에 입력
busan_df3 = pd.read_csv('../../Data_List/v1.0/문화시설 위치/문화시설 위치 통합.csv')
busan_df3['위도'] = pd.NA
busan_df3['경도'] = pd.NA
pd.set_option('display.max_rows', None)

for idx, addr in enumerate(busan_df3['주소']):
    apiurl = 'https://api.vworld.kr/req/address?'
    params = {
        "service": "address",
        "request": "getcoord",
        "crs": "epsg:4326",
        "address": addr,
        "format": "json",
        "type": "parcel",
        "key": apikey,
    }
    response = requests.get(apiurl, params=params, timeout=2)
    time.sleep(random.uniform(1, 2))  # 1~2초 대기
    if response.status_code == 200:
        response_json = response.json()
        if 'response' in response_json and 'result' in response_json['response'] and 'point' in response_json['response']['result']:
            point = response_json['response']['result']['point']
            busan_df3.at[idx, '위도'] = point['y']
            busan_df3.at[idx, '경도'] = point['x']
            print(f"인덱스 {idx}, 주소 '{addr}' 출력 성공: 위도 {point['y']}, 경도 {point['x']}")
        else:
            print(f"인덱스 {idx}, 주소 '{addr}' 출력 실패. 재시도")
            # 재시도 (type="parcel")
            params['type'] = 'road'
            response = requests.get(apiurl, params=params, timeout=2)
            time.sleep(random.uniform(1, 2))  # 1~2초 대기
            if response.status_code == 200:
                response_json = response.json()
                if 'response' in response_json and 'result' in response_json['response'] and 'point' in response_json['response']['result']:
                    point = response_json['response']['result']['point']
                    busan_df3.at[idx, '위도'] = point['y']
                    busan_df3.at[idx, '경도'] = point['x']
                    print(f"인덱스 {idx}, 주소 '{addr}' 재시도 성공: 위도 {point['y']}, 경도 {point['x']}")
                    # 출력 후 '재시도도' 오타를 '재시도'로 수정
                else:
                    print(f"인덱스 {idx}, 주소 '{addr}' 출력 실패")
            else:
                print(f"인덱스 {idx}, 주소 '{addr}' 재시도 요청 실패: 상태 코드 {response.status_code}")
    else:
        print(f"인덱스 {idx}, 주소 '{addr}' 요청 실패: 상태 코드 {response.status_code}")

# 결과 확인

# CSV로 저장 
busan_df3.to_csv('부산광역시_좌표추가.csv', index=False, encoding='utf-8')

인덱스 0, 주소 '부산광역시 금정구 서부로76번길 5 (서동) (부산광역시시립서동도서관)' 출력 실패. 재시도
인덱스 0, 주소 '부산광역시 금정구 서부로76번길 5 (서동) (부산광역시시립서동도서관)' 재시도 성공: 위도 35.219628961, 경도 129.099326831
인덱스 1, 주소 '부산광역시 동구 홍곡로 53 (수정동) (부산광역시립중앙도서관 수정분관)' 출력 실패. 재시도
인덱스 1, 주소 '부산광역시 동구 홍곡로 53 (수정동) (부산광역시립중앙도서관 수정분관)' 재시도 성공: 위도 35.124795631, 경도 129.040489010
인덱스 2, 주소 '부산광역시 동래구 명안로46번길 35 (명장동) 명안로46번길 35' 출력 실패. 재시도
인덱스 2, 주소 '부산광역시 동래구 명안로46번길 35 (명장동) 명안로46번길 35' 재시도 성공: 위도 35.200850708, 경도 129.106543827
인덱스 3, 주소 '부산광역시 부산진구 동천로 79 (부전동) 부산광역시립부전도서관' 출력 실패. 재시도
인덱스 3, 주소 '부산광역시 부산진구 동천로 79 (부전동) 부산광역시립부전도서관' 재시도 성공: 위도 35.155902588, 경도 129.061952938
인덱스 4, 주소 '부산광역시 부산진구 월드컵대로 462 (초읍동) (초읍동) 부산광역시립시민도서관' 출력 실패. 재시도
인덱스 4, 주소 '부산광역시 부산진구 월드컵대로 462 (초읍동) (초읍동) 부산광역시립시민도서관' 재시도 성공: 위도 35.182806221, 경도 129.050141697
인덱스 5, 주소 '부산광역시 부산진구 가야대로 734 (부전동) 부산글로벌빌리지 행정동 5층 ' 출력 실패. 재시도
인덱스 5, 주소 '부산광역시 부산진구 가야대로 734 (부전동) 부산글로벌빌리지 행정동 5층 ' 재시도 성공: 위도 35.156745057, 경도 129.052320235
인덱스 6, 주소 '부산광역시 북구 백양대로1016번다길 43 (구포동) ' 출력 실패. 재시

In [2]:
busan_df4 = pd.read_csv('부산광역시_좌표추가.csv')
busan_df4

,순번,시도,시군구,주소,이름,연간 이용자 수,종류,위도,경도
0,1,부산광역시,금정구,부산광역시 금정구 서부로76번길 5 (서동) (부산광역시시립서동도서관),부산광역시립서동도서관,"112,667",도서관,35.219629,129.099327
1,2,부산광역시,동구,부산광역시 동구 홍곡로 53 (수정동) (부산광역시립중앙도서관 수정분관),부산광역시립중앙도서관수정분관,"150,606",도서관,35.124796,129.040489
2,3,부산광역시,동래구,부산광역시 동래구 명안로46번길 35 (명장동) 명안로46번길 35,부산광역시립명장도서관,"155,228",도서관,35.200851,129.106544
3,4,부산광역시,부산진구,부산광역시 부산진구 동천로 79 (부전동) 부산광역시립부전도서관,부산광역시립부전도서관,0,도서관,35.155903,129.061953
4,5,부산광역시,부산진구,부산광역시 부산진구 월드컵대로 462 (초읍동) (초읍동) 부산광역시립시민도서관,부산광역시립시민도서관,"698,983",도서관,35.182806,129.050142
5,6,부산광역시,부산진구,부산광역시 부산진구 가야대로 734 (부전동) 부산글로벌빌리지 행정동 5층,부산영어도서관,"89,758",도서관,35.156745,129.052320
6,7,부산광역시,북구,부산광역시 북구 백양대로1016번다길 43 (구포동),부산광역시립구포도서관,"421,775",도서관,35.193980,128.996008
7,8,부산광역시,사하구,부산광역시 사하구 승학로 247 (괴정동) 부산광역시립사하도서관,부산광역시립사하도서관,"173,542",도서관,35.102893,128.990917
8,9,부산광역시,서구,부산광역시 서구 대신공원로 41 (서대신동3가) 부산광역시립구덕도서관,부산광역시립구덕도서관,"169,530",도서관,35.121686,129.016901
9,10,부산광역시,연제구,부산광역시 연제구 고분로191번길 16 (연산동) 부산광역시립연산도서관,부산광역시립연산도서관,"270,740",도서관,35.185950,129.104003


# 더 명확한 주소를 위해 재시도 코드를 사용함.
    - 문화시설 도로명 주소가 해당 위치에 맞지않는 주소로 적힘
    - 주소가 예전 주소로 최신화 되어 있지 않음.
해결책
    - 직접 수정
        ( ex 부산진 문화원 등등)